1) Load the segmentation model

In [32]:
import tensorflow as tf
import os
import sys
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

# Model path
data_root = "/mnt/c/Users/pdeschepper/Desktop/PERSONAL/DeepLearning/ImageSegmentation/Snakes_ImageSegmentation_keras/Vipera_segmentation_dataset/"
model_filename = "Vipera_SegmentationModel_V2_100+50epochs.keras"
model_path = os.path.join(data_root, model_filename)

# Import custom functions from the accessory script
sys.path.append("/mnt/c/Users/pdeschepper/Desktop/PERSONAL/DeepLearning/ImageSegmentation/Snakes_ImageSegmentation_keras/")
from HelperFuncs_IoU_DiceLoss_CombinedLoss import combined_loss, dice_loss, mean_iou

# Load model
loaded_model = load_model(
    model_path,
    custom_objects={
        'combined_loss': combined_loss,
        'dice_loss': dice_loss,
        'mean_iou': mean_iou
    } 
)

print("✅ Model loaded successfully!")
loaded_model.summary()

✅ Model loaded successfully!
Model: "functional_2"
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 512, 512,  │          0 │ input_layer_2[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 512, 512,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (

/home/thesnakeguy/miniconda3/envs/tensorflow/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 30 variables whereas the saved optimizer has 58 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2. Segmenting and Extracting the Snakes


In [ ]:
import cv2 # OpenCV for image manipulation
import numpy as np
import glob

# --- Configuration ---
# NOTE: Set IMAGE_HEIGHT and IMAGE_WIDTH to the dimensions your model was trained on.
IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512

# Define paths
# Assumes your test images are in a subfolder named 'test_images'
image_folder = os.path.join(data_root, "test_images/") 
output_folder = os.path.join(data_root, "Extracted_snakes/")

# Create the output directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)
print(f"Saving extracted snakes to: {output_folder}")

# Find all image files in the folder
image_files = glob.glob(os.path.join(image_folder, "*.jpg")) # Or .png, .jpeg, etc.

# --- Processing Loop ---
for img_path in image_files:
    # 1. Load and Preprocess Image
    original_img = cv2.imread(img_path)
    original_h, original_w, _ = original_img.shape
    
    # Resize for the model and normalize pixel values to [0, 1]
    img_for_model = cv2.resize(original_img, (IMAGE_WIDTH, IMAGE_HEIGHT))
    img_for_model = img_for_model / 255.0
    
    # Add a 'batch' dimension for the model input
    img_batch = np.expand_dims(img_for_model, axis=0)
    
    # 2. Predict the Mask
    predicted_mask = loaded_model.predict(img_batch)[0] # Get the first (and only) mask from the batch
    
    # 3. Post-process the Mask
    # Apply a threshold to get a binary mask (0s and 1s)
    binary_mask = (predicted_mask > 0.5).astype(np.uint8) 
    
    # Resize the mask back to the original image's dimensions
    binary_mask_resized = cv2.resize(binary_mask, (original_w, original_h), interpolation=cv2.INTER_NEAREST)
    
    # 4. Extract the Snake using the Mask
    # Create a 4-channel image (RGBA) to support transparency
    extracted_snake = np.zeros((original_h, original_w, 4), dtype=np.uint8)
    
    # Copy the original image's pixels where the mask is 1 (the snake)
    # The `[:, :, np.newaxis]` part makes the mask broadcast correctly
    extracted_snake[:, :, :3] = cv2.bitwise_and(original_img, original_img, mask=binary_mask_resized)
    
    # Set the alpha (transparency) channel: 255 where snake is, 0 otherwise
    extracted_snake[:, :, 3] = binary_mask_resized * 255

    # 

    # 5. Save the Result
    filename = os.path.basename(img_path)
    output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".png") # Save as PNG for transparency
    cv2.imwrite(output_path, extracted_snake)
    
    print(f"Processed and saved {filename}")

print("✨ All images processed!")

3. Analyzing Colors in R with recolorize


In [ ]:
# First, install the package if you haven't already
# install.packages("recolorize")

library(recolorize)

# 1. Set the path to your extracted images
# Use the same path you defined in the Python script for the output folder.
img_dir <- "/mnt/c/Users/pdeschepper/Desktop/PERSONAL/DeepLearning/ImageSegmentation/Snakes_ImageSegmentation_keras/Vipera_segmentation_dataset/Extracted_snakes/"

# Get a list of all the PNG files
img_files <- list.files(img_dir, pattern = ".png$", full.names = TRUE)

# 2. Analyze a single image as an example
# Let's process the first image in the list
example_img_path <- img_files[1]

# Load the image using readImage. It's crucial to set `alpha = TRUE`
# to ignore the transparent background during color analysis.
img_to_recolor <- readImage(example_img_path, alpha = TRUE)

# Run the main recolorize function to find the N most dominant colors.
# Let's find the 5 main colors (e.g., dark background, light background, spots, etc.)
# `plotting = TRUE` will show you the result visually.
recolor_result <- recolorize(img_to_recolor, n = 5, plotting = TRUE)

# 3. Inspect the results
# You can see the identified color centers (in RGB) and their proportions
print("Color centers (RGB):")
print(recolor_result$centers)

print("Proportion of each color:")
print(recolor_result$sizes)


# 4. (Advanced) Process all images in a loop
# You can use lapply to run this on all your images and store the results.
all_color_data <- lapply(img_files, function(file_path) {
  
  img <- readImage(file_path, alpha = TRUE)
  
  # Run without plotting for batch processing
  result <- recolorize(img, n = 5, plotting = FALSE)
  
  # Return a list containing the file name and the color proportions
  return(list(
    filename = basename(file_path),
    color_proportions = result$sizes
  ))
})

# Now `all_color_data` is a list where each element contains the color
# analysis for one of your extracted snakes.
print(all_color_data[1]) # Print the results for the first snake